In [47]:
import pandas as pd
import numpy as np
import re
import emoji

import pymorphy2
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

In [48]:
data = pd.read_csv('tatarstan_message.csv', sep = ';', encoding= 'utf-8')
print(data.columns)
data = data[['Категория, присвоенная модератором', 'Описание заявки']]
data.columns = ['categories', 'description']
data.head(3)

Index(['Дата подачи заявки', 'Долгота', 'Широта', 'Адрес',
       'Категория, присвоенная модератором', 'Описание заявки',
       'Последний исполнитель', 'id исполнителя', 'Статус заявки'],
      dtype='object')


,categories,description
0,Благоустройство территории,На трассе Нижнекамск-Чистополь в лесополосе у ...
1,Благоустройство территории,Во дворе дома Ленинградская 29 отсутствуют пар...
2,Содержание и ремонт муниципальных дорог,Добрый день! Не осуществляется уборка снега в ...


In [49]:
categories = []
for cat in data.categories:
    if cat not in categories:
        categories.append(cat)
print(categories)
print(len(categories))

['Благоустройство территории', 'Содержание и ремонт муниципальных дорог', 'Поликлиники и больницы', 'Бездомные животные', 'Парки и скверы', 'Мобильная связь', 'Организация дорожного движения', 'Незаконные азартные игры', 'Санитарное состояние', 'Нарушение в наружной рекламе', 'Работа спортивных объектов', 'Капитальный ремонт', 'Общественный транспорт', 'Воздух', 'Садоводческие, огороднические и дачные некоммерческие объединения', 'Детские сады', 'Доступная среда', 'Жилищное строительство', 'Почта', 'Нарушение правил торговли', 'Пенсионное обеспечение', 'Вода', nan, 'Содержание и ремонт федеральных и республиканских дорог', 'Меры социальной поддержки, предоставляемые органами социальной защиты', 'Cвалки', 'Работа учреждений культуры', 'Пособия безработным', 'Социальное обслуживание', 'Объекты культурного наследия', 'Сельское хозяйство', 'Цифровое телевидение', 'Тех.средства и услуги по реабилитации инвалидов за счет средств федерального бюджета', 'Опека, попечительство', 'Ошибки в назва

In [50]:
def preprocess_text(text):
    def replace_three_or_more(text): #remove dublicate of symbol
        pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
        return pattern.sub(r"\1\1", text)

    def emo_to_text(text):  
        emoticon = {'улыбка шутка' : ['\:\)', '\:\-\)', '\:\=\)'],
                    'грусть разочарование' : ['\:\(', '\:\-\(', '\:\=\('],
                    'открытая радость' : ['\=\)', '\=\-\)'],
                    'усмешка хихиканье' : ['\:>', '\:\->', '\:\=>'],
                    'улыбка' : ['\:\]', '\:\-\]', '\:\=\]'],
                    'смех' : ['\:D', '\:\-D', '\:\=D'],
                    'сильный смех' : ['\:DD', '\:\-DD', '\:\=DD'],
                    'сарказм' : ['\:\}', '\:\-\}', '\:\=\}'],
                    'подмигивание заигрывание' : ['\;\)', '\;\-\)', '\;\=\)'],
                    'молчание' : ['\:Х', '\:\-Х', '\:\=Х'],
                    'удивленное разочарование' : ['8\(', '8\-\(', '8\=\('],
                    'изумление' : ['B\-о', 'B\=о'],
                    'радостное удивление' : ['\%\)'],
                    'зевота' : ['\|\-o', '\|\=o'],
                    'смущение' : ['\:S', '\:-S', '\:\=S'],
                    'равнодушие скука' : ['\:\|', '\:\-\|', '\:\=\|'],
                    'недоверие сомнение' : ['\:\/', '\:\-\/', '\:\=\/'],
                    'дразнящее показывание языка' : ['\:ь', '\:\-ь', '\:\=ь'],
                    'поцелуй' : ['\:\*', '\:\^\*', '\:\-Ф'],
                    'шалость дурачество' : ['\:0\)'],
                    'кривая ухмылка' : ['\:7', '\:\-7']}

        for key, value in emoticon.items():
            text = re.sub(r"|".join(value), ' ' + key, text)
        return text

    try:
        text = emo_to_text(text)
        text = emoji.demojize(text, delimiters=("", "")) #преобразует эмодзи-символы в слова
        text = text.lower().replace("ё", "е")
        text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
        text = re.sub('@[^\s]+', 'USER', text)
        text = re.sub('[^a-zA-Zа-яА-Я]+', ' ', text) #удаляем цифры и спецсимволы
        text = replace_three_or_more(text) #удаляет дублирование букв, если число букв более 3
        text = re.sub(' +', ' ', text)
        return text.strip()
    except:
        #print(f'Слово {text} не прошло препроцессинг')
        return ''  

data['description_prep'] = data['description'].apply(preprocess_text)
print(data['description_prep'])

0        на трассе нижнекамск чистополь в лесополосе у ...
1        во дворе дома ленинградская отсутствуют парков...
2        добрый день не осуществляется уборка снега в ц...
3        добрый день не осуществляется вывоз мусора с т...
4        на данном участке проезжей части от ул халитов...
                               ...                        
29017    в сентябре по программе развитие садоводческог...
29018                                                     
29019                                                     
29020                                                     
29021                                                     
Name: description_prep, Length: 29022, dtype: object


In [51]:
#стемминг
morph = pymorphy2.MorphAnalyzer() #по умолчанию русский язык
stemmer = SnowballStemmer("english")
stops = set(stopwords.words("english")) | set(stopwords.words("russian"))

ttext_stem = list()
for row in data['description_prep']:
    words = row.split(' ')
    nf_words = list()
    for word in words:
        try:
            #word = word.encode('utf8mb4')
            ru_word = re.sub('[^а-яА-Я]+', ' ', word)
            en_word = re.sub('[^a-zA-Z]+', ' ', word)
            if ru_word == word and word != '': # кириллица
                parse_word = morph.parse(word)[0]
                nf_word = parse_word.normal_form
                nf_words.append(nf_word.strip())
            elif en_word == word and word != '': # латиница
                nf_word = stemmer.stem(word)
                nf_words.append(nf_word.strip())
            else:
                pass #смешанные слова из кириллицы и латиницы не анализируются
        except Exception as e:
                print(f'Слово не преобразовано {e}') 
                continue
    nf_words = [w for w in nf_words if not w in stops]
    nf_words = ' '.join(nf_words) #ttext_stem
    ttext_stem.append(nf_words)
if ttext_stem != None:
    ttext_stem = pd.Series(ttext_stem) 
    data['description_stem'] = ttext_stem  
    print(data['description_stem'])

0        трасса нижнекамск чистополь лесополоса пгт кам...
1        двор дом ленинградский отсутствовать парковочн...
2        добрый день осуществляться уборка снег центр г...
3        добрый день осуществляться вывоз мусор террито...
4        данный участок проезжий часть ул халитовый дом...
                               ...                        
29017    сентябрь программа развитие садоводческий движ...
29018                                                     
29019                                                     
29020                                                     
29021                                                     
Name: description_stem, Length: 29022, dtype: object


In [52]:
data.to_csv("tatarstan_message_2.csv", sep = ';')